In [2]:
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sys
import os
import logging
import random
import copy
import time
import queue
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

sys.path.append(os.path.abspath(".."))
from Modules.Helper import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotters
from Modules.Equation import Equation

In [3]:
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='../Data/GRN5_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])
t_eval = np.array(df['t'])
original = np.array(df[labels]).T

bounds = {
    'tau': (0.1, 5.0),
    'k': (0.1, 2.0),
    'n': (0, 30.0)
}

IND_SIZE = 19
GENERATION_LIMIT = 5000
NO_IMPROVEMENT_LIMIT = 50
HARD_SIGMA_INCREASE_FACTOR = 10
TOLERANCE = 1E-4

In [4]:
class Individual:
    def __init__(self, method='RK45', error='ABS'):
        # Estrutura do indivíduo contendo os coeficientes para cada variável
        self.coeffs = {
            'A': {
                'E': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'B': {
                'A': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'C': {
                'B': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'E': {
                'D': {'n': None, 'k': None, '-': False},
                'B': {'n': None, 'k': None, '-': False},
                'E': {'n': None, 'k': None, '-': False},
                'tau': None,
            }
        }
        
        self.ind_size = IND_SIZE
        self.fitness = np.inf
        self.method = method
        self.error = error
        
    @staticmethod
    def get_bounds_list():
        bounds_list = []
        gene_regulators = {
            'A': ['E'],
            'B': ['A'],
            'C': ['B'],
            'D': ['C'],
            'E': ['D', 'B', 'E']
        }
        for gene in ['A', 'B', 'C', 'D', 'E']:
            bounds_list.append(bounds['tau'])
            for _ in gene_regulators[gene]:
                bounds_list.append(bounds['n'])
                bounds_list.append(bounds['k'])
        return bounds_list
     
    @staticmethod
    def list_to_ind(list_ind, method, error):
        i = 0
        ind = Individual(method=method, error=error)
        
        # Mapeamento dos reguladores para cada gene
        gene_regulators = {
            'A': ['E'],
            'B': ['A'],
            'C': ['B'],
            'D': ['C'],
            'E': ['D', 'B', 'E']  # Assumindo 3 reguladores para E
        }
        
        for gene in ['A', 'B', 'C', 'D', 'E']:
            # Atribui tau
            ind.coeffs[gene]['tau'] = list_ind[i]
            i += 1
            
            # Atribui n e k para cada regulador
            for regulator in gene_regulators[gene]:
                ind.coeffs[gene][regulator]['n'] = list_ind[i]
                ind.coeffs[gene][regulator]['k'] = list_ind[i+1]
                i += 2
                
        return ind

    def ind_to_list(self):
        ind_list = []
        for gene in ['A', 'B', 'C', 'D', 'E']:
            ind_list.append(self.coeffs[gene]['tau'])
            
            regulators = [k for k in self.coeffs[gene].keys() if k != 'tau']
            for regulator in regulators:
                ind_list.append(self.coeffs[gene][regulator]['n'])
                ind_list.append(self.coeffs[gene][regulator]['k'])
                
        return ind_list
    
    @staticmethod
    def apply_bounds(population, method, error):
        for ind in population:
            list_ind = Individual.list_to_ind(ind, method, error)
            ind[:] = Individual.ind_to_list(list_ind)
    
    @staticmethod    
    def cma_evaluate(method, error, list_ind):
        ind = Individual.list_to_ind(list_ind, method, error)
        ind.calc_fitness()
        return ind.fitness,
    
    # Calcula o fitness do indivíduo simulando o sistema de ODEs
    def calc_fitness(self):
        try:
            equation = Equation(self.numerical_coeffs, labels)
            # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
            y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
            
            if self.error == 'ABS':
                self.fitness = self.abs_error(original, y)
            elif self.error == 'MSE':
                self.fitness = self.MSE_error(original, y)
            elif self.error == 'MABS':
                self.fitness = self.mean_abs_error(original, y)
            elif self.error == 'SQUARED':
                self.fitness = self.squared_error(original, y)
            
            self.fitness = min(self.fitness, 1e6)
        except:
            # Trata exceções relacionadas ao solver
            print("Fitness Overflow")
            self.fitness = 1e6
            
    def calc_all_fitness(self):
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        
        return {
            'ABS_Fitness': self.abs_error(original, y), 
            'SQUARED_Fitness': self.squared_error(original, y),
            'MSE_Fitness': self.MSE_error(original, y), 
            'MABS_Fitness': self.mean_abs_error(original, y), 
        }
     
       
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals
        
        dA = equation.full_eq(vals, 'A', 'E')
        dB = equation.full_eq(vals, 'B', 'A')
        dC = equation.full_eq(vals, 'C', 'B')
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+B', '+D'], ['+D', '+E']])

        return [dA, dB, dC, dD, dE]
    
    @staticmethod
    def abs_error(original, pred):
        return sum(sum(abs(original-pred)))
    
    @staticmethod
    def squared_error(original, pred):
        return sum(sum( (original-pred)**2 ))**(1/2)
    
    @staticmethod
    def MSE_error(original, pred):
        return np.mean((original-pred)**2)
    
    @staticmethod
    def mean_abs_error(original, pred):
        return np.mean(abs(original-pred))
            
    @staticmethod
    def initialize_ind():
        ind = Individual()
        gene_regulators = {
            'A': ['E'],
            'B': ['A'],
            'C': ['B'],
            'D': ['C'],
            'E': ['D', 'B', 'E']
        }
        
        for gene in ['A', 'B', 'C', 'D', 'E']:
            # Gera valor para tau
            ind.coeffs[gene]['tau'] = random.uniform(*bounds['tau'])
            
            # Gera valores para n e k de cada regulador
            for regulator in gene_regulators[gene]:
                ind.coeffs[gene][regulator]['n'] = random.uniform(*bounds['n'])
                ind.coeffs[gene][regulator]['k'] = random.uniform(*bounds['k'])
        
        ind.calc_fitness()
        return ind
    
    @property
    def numerical_coeffs(self):
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for gene in numerical_coeffs:
            numerical_coeffs[gene]['tau'] = self.coeffs[gene]['tau']
            for regulator in numerical_coeffs[gene]:
                if regulator != 'tau':
                    numerical_coeffs[gene][regulator]['n'] = int(self.coeffs[gene][regulator]['n'])
                    numerical_coeffs[gene][regulator]['k'] = self.coeffs[gene][regulator]['k']
        return numerical_coeffs
    
    def plot(self, method='RK45', comparison=True):
        methods = [self.method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        Plotters.plot_methods(results=results,t=t_eval, methods=methods, labels=labels)
        if comparison:
            Plotters.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)
            
    @staticmethod        
    def initialize_average_bounds(bounds, ind_size):
        averages = [np.mean(value) for value in bounds.values()]
        array = np.resize(averages, ind_size)
        return array   
        
    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual (fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"

In [ ]:
def run_evolution(seed, error, method, result_container, timeout_limit):
    logging.info(f"Starting DE execution: Solver={method}, Error={error}, Seed={seed}")
    
    bounds_list = Individual.get_bounds_list()
    np.random.seed(seed)
    
    def objective_function(params):
        ind = Individual.list_to_ind(params, method, error)
        ind.calc_fitness()
        return ind.fitness
    
    start_time = time.time()
    
    try:
        result = differential_evolution(
            objective_function,
            bounds_list,
            strategy='best1bin',
            maxiter=1000,
            popsize=15,
            mutation=0.8,
            recombination=0.9,
            seed=seed,
            polish=False,
            callback=lambda x, convergence: time.time() - start_time > timeout_limit,
            disp=True
        )
        
        best_ind = Individual.list_to_ind(result.x, method, error)
        best_ind.calc_fitness()
        
        # Debug: Mostra resultados parciais
        print(f"\nSeed {seed} ({error}) - Melhor fitness: {best_ind.fitness:.4f}")
        print("Parâmetros:", best_ind.ind_to_list())
        
    except Exception as e:
        logging.error(f"DE Error: {str(e)}")
        best_ind = None

    finally:
        execution_time = time.time() - start_time
        
        errors_result = best_ind.calc_all_fitness() if best_ind else {
            'ABS_Fitness': None, 'SQUARED_Fitness': None, 
            'MSE_Fitness': None, 'MABS_Fitness': None
        }

        result_data = {
            'best_ind': best_ind,
            'error_type': error,
            'method': method,
            'seed': seed,
            'ABS_Fitness': errors_result['ABS_Fitness'],
            'SQUARED_Fitness': errors_result['SQUARED_Fitness'],
            'MSE_Fitness': errors_result['MSE_Fitness'],
            'MABS_Fitness': errors_result['MABS_Fitness'],
            'execution_time': execution_time,
        }

        result_container.append(result_data)
        logging.info(f"Execution finished for {method}, seed {seed}, error {error} in {execution_time:.2f} seconds.")
        
CSV_FILENAME = 'GRN5_DE.csv'

if os.path.exists(CSV_FILENAME):
    results_df = pd.read_csv(CSV_FILENAME)
    existing_results = results_df[['seed', 'error_type', 'method']]
else:
    results_df = pd.DataFrame(columns=[
        'best_ind', 'error_type', 'method', 'seed', 'ABS_Fitness',
        'SQUARED_Fitness', 'MSE_Fitness', 'MABS_Fitness', 'execution_time'
    ])
    existing_results = pd.DataFrame(columns=['seed', 'error_type', 'method'])

all_methods = ['RK45', 'DOP853', 'LSODA']
errors = ['ABS', 'SQUARED', 'MSE', 'MABS']
seeds = 10
TIMEOUT_LIMIT = 60*60  # 1 hora

# Thread-safe queue e paralelização
result_queue = queue.Queue()
write_lock = threading.Lock()

def process_and_save_results():
    global results_df
    while True:
        result = result_queue.get()
        if result is None:
            break
            
        with write_lock:
            # Converter o indivíduo para string serializável
            if result['best_ind'] is not None:
                result['best_ind'] = str(result['best_ind'].ind_to_list())
            
            results_df = pd.concat([results_df, pd.DataFrame([result])], ignore_index=True)
            results_df.to_csv(CSV_FILENAME, index=False)

def process_seed_error_method(seed, error, method):
    if not existing_results[
        (existing_results['seed'] == seed) &
        (existing_results['error_type'] == error) &
        (existing_results['method'] == method)
    ].empty:
        return

    result_container = []
    try:
        run_evolution(seed, error, method, result_container, TIMEOUT_LIMIT)
    except Exception as e:
        logging.error(f"Critical error: {str(e)}")
    
    if result_container:
        result_queue.put(result_container[0])

# Iniciar thread de escrita
writer_thread = threading.Thread(target=process_and_save_results, daemon=True)
writer_thread.start()

# Execução paralela
with ThreadPoolExecutor(max_workers=4) as executor:  # Reduzir workers para GRN complexo
    futures = []
    for seed in range(seeds):
        for error in errors:
            for method in all_methods:
                futures.append(executor.submit(process_seed_error_method, seed, error, method))
    
    for future in as_completed(futures):
        future.result()

# Finalizar
result_queue.put(None)
writer_thread.join()

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 1: f(x)= 135.534
differential_evolution step 2: f(x)= 135.534
differential_evolution step 3: f(x)= 135.534
differential_evolution step 4: f(x)= 133.247
differential_evolution step 5: f(x)= 133.247
differential_evolution step 6: f(x)= 133.247
differential_evolution step 7: f(x)= 129.466
differential_evolution step 8: f(x)= 129.466
differential_evolution step 1: f(x)= 135.529
differential_evolution step 1: f(x)= 135.533
differential_evolution step 9: f(x)= 129.466
differential_evolution step 1: f(x)= 10.9789
differential_evolution step 10: f(x)= 116.573
differential_evolution step 11: f(x)= 116.573
differential_evolution step 12: f(x)= 116.573
differential_evolution step 13: f(x)= 116.573
differential_evolution step 2: f(x)= 135.529
differential_evolution step 2: f(x)= 135.533
differential_evolution step 14: f(x)= 116.573
differential_evolution step 2: f(x)= 10.9789
differential_evolution step 15: f(x)= 109.322
differential_evolution step 16: f(x)= 109.322
dif

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 18: f(x)= 8.49575
differential_evolution step 24: f(x)= 119.452
differential_evolution step 21: f(x)= 109.318
differential_evolution step 25: f(x)= 119.452
differential_evolution step 19: f(x)= 8.49575
differential_evolution step 22: f(x)= 109.318
differential_evolution step 26: f(x)= 119.452
differential_evolution step 23: f(x)= 109.318
differential_evolution step 20: f(x)= 8.49575
differential_evolution step 1: f(x)= 10.9781
differential_evolution step 27: f(x)= 119.232
differential_evolution step 24: f(x)= 102.017
differential_evolution step 28: f(x)= 117.392
differential_evolution step 21: f(x)= 8.49575
differential_evolution step 2: f(x)= 10.9781
differential_evolution step 29: f(x)= 117.392
differential_evolution step 25: f(x)= 102.017
differential_evolution step 22: f(x)= 8.49575
differential_evolution step 30: f(x)= 107.398
differential_evolution step 26: f(x)= 102.017
differential_evolution step 3: f(x)= 10.1607
differential_evolution step 31: f(x)=

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


...

differential_evolution step 6: f(x)= 10.0084
differential_evolution step 36: f(x)= 102.894
differential_evolution step 52: f(x)= 98.7564
differential_evolution step 7: f(x)= 10.0084
differential_evolution step 37: f(x)= 102.894
differential_evolution step 8: f(x)= 10.0084
differential_evolution step 1: f(x)= 10.0009
differential_evolution step 53: f(x)= 98.7564
differential_evolution step 38: f(x)= 102.894
differential_evolution step 2: f(x)= 10.0009
differential_evolution step 9: f(x)= 9.60944
differential_evolution step 54: f(x)= 98.7564
differential_evolution step 39: f(x)= 102.894
differential_evolution step 3: f(x)= 10.0009
differential_evolution step 10: f(x)= 9.60944
differential_evolution step 40: f(x)= 102.894
differential_evolution step 55: f(x)= 98.7564
differential_evolution step 4: f(x)= 10.0009
differential_evolution step 11: f(x)= 9.37297
differential_evolution step 41: f(x)= 102.894
differential_evolution step 5: f(x)= 10.0009
differential_evolution step 56: f(x)= 98.75

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 155: f(x)= 96.4558
differential_evolution step 1: f(x)= 10.0083
differential_evolution step 2: f(x)= 10.0083
differential_evolution step 3: f(x)= 10.0083
differential_evolution step 110: f(x)= 8.06736
differential_evolution step 4: f(x)= 10.0083
differential_evolution step 120: f(x)= 7.99628
differential_evolution step 5: f(x)= 10.0083
differential_evolution step 156: f(x)= 96.4558
differential_evolution step 6: f(x)= 10.0083
differential_evolution step 7: f(x)= 10.0083
differential_evolution step 8: f(x)= 10.0083
differential_evolution step 111: f(x)= 8.06736
differential_evolution step 9: f(x)= 9.61003
differential_evolution step 10: f(x)= 9.61003
differential_evolution step 157: f(x)= 96.4558
differential_evolution step 11: f(x)= 9.37323
differential_evolution step 121: f(x)= 7.99628
differential_evolution step 12: f(x)= 9.37323
differential_evolution step 13: f(x)= 9.37323
differential_evolution step 112: f(x)= 8.06736
differential_evolution step 14: f(x

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 16: f(x)= 9.37323
differential_evolution step 17: f(x)= 9.37323
differential_evolution step 122: f(x)= 7.99628
differential_evolution step 18: f(x)= 9.37323
differential_evolution step 113: f(x)= 8.06736
differential_evolution step 19: f(x)= 9.08857
differential_evolution step 20: f(x)= 9.08857
differential_evolution step 21: f(x)= 9.08857
differential_evolution step 22: f(x)= 9.08857
differential_evolution step 23: f(x)= 9.08857
differential_evolution step 114: f(x)= 8.06736
differential_evolution step 24: f(x)= 9.08857
differential_evolution step 1: f(x)= 0.400676
differential_evolution step 123: f(x)= 7.96757
differential_evolution step 25: f(x)= 9.04627
differential_evolution step 26: f(x)= 8.77135
differential_evolution step 27: f(x)= 8.77135
differential_evolution step 28: f(x)= 8.77135
differential_evolution step 115: f(x)= 8.06736
differential_evolution step 2: f(x)= 0.400676
differential_evolution step 29: f(x)= 8.77135
differential_evolution step 3

In [ ]:
logging.basicConfig(
    filename="GRN5_DE_execution.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)